In [ ]:
# transform sal.json
import json

with open('./sal.json', 'r') as f:
  data = json.load(f)

sal = {key:value["gcc"] for (key,value) in data.items()}
#print(sal)

In [ ]:
set(sal.values())

In [ ]:
# load test_1.json
with open('./test_1.json', 'r', encoding='UTF-8') as f:
  t1_data = json.load(f)

print(t1_data)

In [ ]:
# load test_2.json
with open('./test_2.json', 'r', encoding='UTF-8') as f:
  t2_data = json.load(f)

print(t2_data)

In [ ]:
from thefuzz import fuzz
from thefuzz import process

# Get geoinfo (with comma, get the first bit; without comma, check with full text)
# Check against 

def get_prob(data, geo_data):
    m_key = ""
    m_prob = 0
    m_val = ""
    geo_info = data["includes"]["places"][0]["full_name"].lower().split(",", 1)[0]
    #print(geo_info)
    
    if geo_info == "australia":
        print(geo_info)
        print("", 0, "")
    
    
    for key, val in geo_data.items():
        if geo_info in key: 
            # check if the val is different to the m_val. if so, determine which one is more accurate with the full geo_info
            if m_val and m_val != val:
                print(geo_info)
                print("    GCC changed")
                print(m_key, m_prob, geo_data.get(m_key))
            cur_prob = fuzz.ratio(key, geo_info)
            if m_prob < cur_prob:
                m_prob = cur_prob
                m_key = key
                m_val = val
    #print(m_key, m_prob, geo_data.get(m_key))

get_prob(t1_data, sal)

get_prob(t2_data, sal)

In [ ]:
with open('./twitter-data-small.json', 'r', encoding='UTF-8') as f:
  full_data = json.load(f)

location_set = set()
for data in full_data:
    location_set.add(data["includes"]["places"][0]["full_name"])
    get_prob(data, sal)
    


In [ ]:
# full_name": "sydney
# , sydney
location_set

### Geo Location Scenarios 
1. One word: 'Murwillumbah', 'Australia'   
    Check them straightaway; Ignore Australia
2. Two words:  Split it and get the first bit and the second bit  
2.1. Two words - Starts with State: 'Victoria, Australia', 'South Australia, Australia',
    Check the second bit, if it's Australia, ignore it
    
2.2. Two words - Starts with GCC: 'Sydney, New South Wales'
    Check the first bit with a pre-defined dictionary for GCCs, if hit, return; if not carry on.

3.3. Two words - Starts with suburb: 'Picton, New South Wales', 'Rockhampton, Queensland'
    Check if the first bit belongs to any suburb. If not, ignore.
    If mathches with one suburb, return;
    If matches with multiple,
        check if there is an exact match, if so, return it
        check if all suburbs have the same gcc values, if so, check if the gcc is of any GCC. 
            If so, return; if not, ignore.
        If not, try to use the second bit to determine where it is from. 
            convert the second bit to abb. e.g. New South Wales should be matched to nsw, Queensland should be matched to qld
            Check if the abb is in the suburb name or gcc.
                If get one match, return it
                If get multiple matches, flag it
                If get no match at all, ignore it

In [ ]:
dict_t = {"vvv": (1, 2), "ggg": (1,1), "bbb": (5,1), "aaa": (11,3)}
sorted_dict = sorted(dict_t.items(), key=lambda x: (-x[1][0], -x[1][1]))
print(sorted_dict)

In [ ]:
import json
sal_file = open('./sal.json')
sal = json.load(sal_file)


def get_place_code(sal):
    place_code_lst = [[] for i in range(7)]

    for (key, value) in sal.items():
        code = value.get("gcc")

        if code[1] == 'g':
            index = int(value.get("ste"))
            place_code_lst[index - 1].append({key: code})

    # for i in range(7):
    #     print(place_code_lst[i])
    #     print("\n")

    return place_code_lst

get_place_code(sal)

In [ ]:
import numpy as np
import pandas as pd

def sum(a, b):
    return a+b

def sum_all(*args):
    count = 0
    for arg in args:
        count += arg
    return count

author_list = [f"author_{i}" for i in range(10)]
# print(author_list)
np_array = np.random.binomial(n=10, p=0.2, size=(10,2))
df = pd.DataFrame(np_array, index= pd.Index(author_list, name="Authors:"), columns=pd.Index(['1gsyd', '2gmelb'], name='GGC:'))

df
# print(df[0])
# print(df[1])


In [ ]:
print("#Twitter pre Author")
# np.vectorize(sum_all)(df["1gsyd"],df["2gmelb"])
%timeit np.vectorize(sum_all)(df["1gsyd"],df["2gmelb"])

In [ ]:
# Preferred approach

print("#Twitter pre Author")
# np.sum(np_array, axis=0)
%timeit np.sum(np_array, axis=0)

In [ ]:
print("#Twitter pre Author")
np.sum(np_array, axis=1)
# %timeit np.sum(np_array, axis=0)

In [ ]:
print("#Twitter pre Author")
# t_df.sum()
%timeit t_df.sum()

In [ ]:
t_df = df.T
print("#Twitter pre GCC")
print(t_df)
out = np.vectorize(sum_all)(t_df["author_0"],t_df["author_1"],t_df["author_2"], t_df["author_3"])
print(out)
# %timeit np.vectorize(sum)(df["1gsyd"],df["2gmelb"])

In [ ]:
%timeit np.power(df[0] + df[1],1)